In [1]:
import torch
import numpy as np
from customs.models import DecisionTransformer02
import torch.nn as nn
import numpy as np
from customs.data import create_sequences, normalize_data, denormalize_data, SequenceDataset
from torch.utils.data import Dataset, DataLoader


/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

variant = {}
variant["state_dim"] = 1
variant["state_range"] = np.array([-1.0, 1.0])
variant["K"] = 4
MAX_EPISODE_LEN = 1000
variant["embed_dim"] = 512
variant["n_layer"] = 4
variant["n_head"] = 4
variant["activation_function"] = "gelu"
variant["dropout"] = 0.1
variant["ordering"] = 0
variant["init_temperature"] = 0.1
variant["target_entropy"] = -variant["state_dim"]
variant["eval_context_length"] = 5
variant["warmup_steps"] = 1000
variant["learning_rate"] = 1e-4
variant["weight_decay"] = 5e-4
variant["stocastic_policy"] = False
variant["telebot"] = False

variant["dataset_num_squence"] = 1000
variant["dataset_seq_length"] = 5
variant["model_seq_length"] = 5 - 1
variant["train_total_epoch"] = 1000


In [3]:
# 시퀀스 데이터 생성 및 데이터셋 객체 생성
train_sequences = create_sequences(seq_length=variant['dataset_seq_length'], num_sequences=variant["dataset_num_squence"], start=-2.0, end=5.0, step=0.01)
normalized_train_sequences = normalize_data(train_sequences, -2.0, 5.0, -1.0, 1.0)
denormalized_train_data = denormalize_data(normalized_train_sequences, -2.0, 5.0, -1.0, 1.0)
dataset = SequenceDataset(train_sequences)

# 데이터셋 예시 출력
print("시퀀스 데이터 예시:", train_sequences[0])
print("normalized seq data:", normalized_train_sequences[0])
print("denormalized seq data:", denormalized_train_data[0])
print("데이터셋 크기:", len(dataset))


시퀀스 데이터 예시: [2.58843598 2.59843598 2.60843598 2.61843598 2.62843598]
normalized seq data: [0.31098171 0.31383885 0.31669599 0.31955314 0.32241028]
denormalized seq data: [2.58843598 2.59843598 2.60843598 2.61843598 2.62843598]
데이터셋 크기: 1000


In [4]:
import time
from lamb import Lamb
import wandb
from jesnk_utils.utils import get_current_time
import asyncio
from customs.trainer import loss_fn_stochastic, loss_fn_deterministic, train

# 데이터 로더 설정
batch_size = 1024
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = DecisionTransformer02(
    state_dim=variant["state_dim"],
    state_range= variant["state_range"],
    max_length=variant["K"],
    eval_context_length=variant["eval_context_length"],
    max_ep_len=MAX_EPISODE_LEN,
    hidden_size=variant["embed_dim"],
    n_layer=variant["n_layer"],
    n_head=variant["n_head"],
    n_inner=4 * variant["embed_dim"],
    activation_function=variant["activation_function"],
    n_positions=1024,
    resid_pdrop=variant["dropout"],
    attn_pdrop=variant["dropout"],
    stochastic_policy=variant["stocastic_policy"],
    ordering=variant["ordering"],
    init_temperature=variant["init_temperature"],
    target_entropy=variant["target_entropy"],
).to(device=device)
    

optimizer = Lamb(
            model.parameters(),
            lr=variant["learning_rate"],
            weight_decay=variant["weight_decay"],
            eps=1e-8,
)
scheduler = torch.optim.lr_scheduler.LambdaLR(
            optimizer, lambda steps: min((steps + 1) / variant["warmup_steps"], 1)
)


current_time = get_current_time()
wandb_enable = False
telebot_enable = False
if wandb_enable :
    wandb.init(project="GPT_exp", entity="jesnk", name=f"DT02_DET_{current_time}")
    wandb.config.update(variant)
    wandb.watch(model)

model.to(device)
loss_fn = loss_fn_stochastic if variant["stocastic_policy"] else loss_fn_deterministic
last_epoch_token_error = train(model, variant, train_loader, loss_fn, optimizer, scheduler)
#last_epoch_token_error = str(round(last_epoch_token_error, 9))

Epoch [1/1000], Loss: 7.5352507, token error : 2.297421455383301Time: 0.1931s
Epoch [2/1000], Loss: 7.5188479, token error : 2.294431686401367Time: 0.1701s
Epoch [3/1000], Loss: 7.5177474, token error : 2.2963950634002686Time: 0.1663s
Epoch [4/1000], Loss: 7.5265765, token error : 2.297307252883911Time: 0.1635s
Epoch [5/1000], Loss: 7.5353703, token error : 2.298689126968384Time: 0.1649s
Epoch [6/1000], Loss: 7.5275760, token error : 2.294843912124634Time: 0.1633s
Epoch [7/1000], Loss: 7.5166440, token error : 2.2939229011535645Time: 0.1574s
Epoch [8/1000], Loss: 7.5445242, token error : 2.297508955001831Time: 0.1564s
Epoch [9/1000], Loss: 7.5032248, token error : 2.2912540435791016Time: 0.1627s
Epoch [10/1000], Loss: 7.5122790, token error : 2.2947752475738525Time: 0.1629s
Epoch [11/1000], Loss: 7.4908781, token error : 2.2899391651153564Time: 0.1655s
Epoch [12/1000], Loss: 7.5258880, token error : 2.294064998626709Time: 0.2337s
Epoch [13/1000], Loss: 7.4972229, token error : 2.291516

RuntimeError: [enforce fail at inline_container.cc:274] . unexpected pos 55641664 vs 55641552

In [ ]:
model.load("./gpt2_ed512_nh4_nl4_sdl5_ns500000_lr0.0001_g0.0005_epoch1000_tte1.599798321723938_ep540.pt")

DecisionTransformer02(
  (transformer): GPT2Model(
    (wte): Embedding(1, 512)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, eleme

In [ ]:
def test(model, test_loader, stochastic_policy=False):
    model.eval()
    total_loss = 0
    total_token_error = []
    count = 0
    with torch.no_grad():
        for _, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            #print(inputs.shape, targets.shape)
            outputs = model(inputs.unsqueeze(-1))
            padding_mask = torch.ones((inputs.shape[0], seq_length), dtype=torch.long)
            print(f"inputs : {inputs[0]}")
            print(f"targets : {targets[0]}")
            print(f"outputs : {outputs[0].detach().cpu().squeeze()}")
            if stochastic_policy :
                loss, nll, entropy = loss_fn(outputs, targets.unsqueeze(-1),attention_mask=padding_mask,entropy_reg=model.temperature().detach())
            else :
                loss = loss_fn(outputs, targets.unsqueeze(-1),attention_mask=padding_mask)
            total_loss += loss.item()
            
            if stochastic_policy:
                output_sample = outputs.mean
            else :
                output_sample = outputs
            token_error = torch.abs(targets.detach() - output_sample.detach().reshape(targets.shape)).mean()
            print("="*20)

            total_token_error.append(token_error.cpu())
            count += 1
    avg_loss = total_loss / len(test_loader)
    avg_token_error = np.mean(total_token_error)
    print(f"Test Loss: {avg_loss:.4f}")
    print(f"Test Token Error: {avg_token_error:.4f}")
    print(f"count : {count}")

# 테스트 데이터 생성 및 데이터셋 객체 생성
test_sequences = create_sequences(seq_length=seq_data_length, num_sequences=100000, start= -5.0, end=5.0)  # 예: 200개의 테스트 시퀀스 생성
test_dataset = SequenceDataset(test_sequences)

# 데이터 로더 설정
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# 손실 함수와 옵티마이저

# 테스트 실행
test(model, test_loader)


inputs : tensor([2.7575, 2.7675, 2.7775, 2.7875], device='cuda:0')
targets : tensor([2.7675, 2.7775, 2.7875, 2.7975], device='cuda:0')
outputs : tensor([1., 1., 1., 1.])
inputs : tensor([-4.1811, -4.1711, -4.1611, -4.1511], device='cuda:0')
targets : tensor([-4.1711, -4.1611, -4.1511, -4.1411], device='cuda:0')
outputs : tensor([-1., -1., -1., -1.])
inputs : tensor([-4.1425, -4.1325, -4.1225, -4.1125], device='cuda:0')
targets : tensor([-4.1325, -4.1225, -4.1125, -4.1025], device='cuda:0')
outputs : tensor([-1., -1., -1., -1.])
inputs : tensor([3.5357, 3.5457, 3.5557, 3.5657], device='cuda:0')
targets : tensor([3.5457, 3.5557, 3.5657, 3.5757], device='cuda:0')
outputs : tensor([1., 1., 1., 1.])
inputs : tensor([-1.5597, -1.5497, -1.5397, -1.5297], device='cuda:0')
targets : tensor([-1.5497, -1.5397, -1.5297, -1.5197], device='cuda:0')
outputs : tensor([-1., -1., -1., -1.])
inputs : tensor([3.6347, 3.6447, 3.6547, 3.6647], device='cuda:0')
targets : tensor([3.6447, 3.6547, 3.6647, 3.674